In [0]:
df_json = spark.read.format('json').option('inferSchema',True)\
                    .option('header',True)\
                    .option('multiLine',False)\
                    .load('/Volumes/workspace/default/daya/drivers.json')
df_json.display()

In [0]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('/Volumes/workspace/default/daya/BigMart Sales.csv')
df.display()

COLLECTION LIST

In [0]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.display()

In [0]:
from pyspark.sql.functions import *
df_book.groupBy('user').agg(collect_list('book')).display() 

PIVOT

In [0]:
df.select('Item_Type','Outlet_Size','Item_MRP').display()

In [0]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).display()

When-Otherwise(case statement)

scinario - 1

In [0]:
df=df.withColumn('veg_flag',when(col('Item_Type')=='Meat','non-veg').otherwise ('veg'))
df.display() 

In [0]:
df.withColumn('veg_exp_flag',when(((col('veg_flag')=='veg') & (col('Item_MRP')<100)),'Veg_Inexpensive').when((col('veg_flag')=='veg') & (col('Item_MRP')>100),'Veg_Expensive').otherwise('Non_Veg')).display() 

In [0]:
df.withColumn('veg_exp',when(((col('veg_flag')=='veg') &(col('Item_MRP')<100)),'basic_price')\
    .when((col('veg_flag')=='veg') &(col('Item_MRP')>100),'expensive_price')\
        .otherwise('non_veg')).display() 

JOINS

In [0]:
dataj1 = [('1','gaur','d01','L&T'),
          ('2','kit','d02','L&T'),
          ('3','sam','d03','T&L'),
          ('4','tim','d03','L&T'),
          ('5','aman','d05','L&T'),
          ('6','nad','d06','L&T'),] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING,company STRING'
df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR','L&T'),
          ('d02','Marketing','L&T'),
          ('d03','Accounts','L&T'),
          ('d04','IT','L&T'),
          ('d05','Finance','L&T'),]

schemaj2 = 'dept_id STRING, department STRING,company STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [0]:
df1.display()
df2.display()

INNER jOIN

In [0]:
df1.join(df2,df1.dept_id==df2.dept_id,'inner').display()

In [0]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').display()

In [0]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').display()

In [0]:
df1.join(df2,[df1.dept_id==df2.dept_id,df1.company==df2.company],'inner').display()

LEFT JOIN

In [0]:
df1.join(df2,[df1.dept_id==df2.dept_id,df1.company==df2.company],'left').display()

RIGHT JOIN

In [0]:
df1.join(df2,[df1.dept_id==df2.dept_id,df1.company==df2.company],'right').display()

ANTI JOIN

In [0]:
df1.join(df2,[df1.dept_id==df2.dept_id,df1.company==df2.company],'anti').display()

WINDOW FUNCTIONS

In [0]:
from pyspark.sql.window import Window

In [0]:
df.display()

ROW_NUMBER

In [0]:
df.withColumn('rowcol',row_number().over(Window.orderBy(col('Item_Identifier').desc()))).display()   

RANK

In [0]:
df.withColumn('RANKCOL',rank().over(Window.orderBy(col('Item_Identifier').desc()))).display() 

DENSE RANK

In [0]:
df.withColumn('RANKCOL',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).display() 

In [0]:
df.withColumn('RANKCOL',dense_rank().over(
    Window.partitionBy('Outlet_Identifier','Outlet_Type')
              .orderBy(col('Item_Identifier').desc())
)).display() 


RANK VS DENSE RANK

In [0]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc())))\
        .withColumn('denseRank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).display()

CUMULATIVE SUM(adding row1+row2,etcc. next line )

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).display()

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()
     

In [0]:
df.withColumn('totalsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).display()

USER DEFINED FUNCTIONS (UDF)

STEP1

In [0]:
def my_fun(x):
    return x*x 

In [0]:
udf_fun=udf(my_fun)

In [0]:
df.withColumn('mynewcol',udf_fun('Item_MRP')).display()

- DATA WRITING

.APPEND
.OVERWRITE
.ERROR
.IGNORE

In [0]:
#'/Volumes/workspace/default/daya/drivers.json'

CSV

In [0]:
df.write.format('csv')\
        .save('/Volumes/workspace/default/daya/csv1/data.csv')


APPEND

In [0]:
df.write.format('csv')\
        .mode('append')\
        .save('/Volumes/workspace/default/daya/csv/data.csv')

In [0]:
df.write.format('csv')\
        .mode('append')\
        .option('path','/Volumes/workspace/default/daya/csv/data.csv')\
        .save()


Overwrite

In [0]:
df.write.format('csv')\
        .mode('overwrite')\
        .option('path','/Volumes/workspace/default/daya/csv/data.csv')\
        .save()

Error

In [0]:
df.write.format('csv')\
        .mode('error')\
        .option('path','/Volumes/workspace/default/daya/csv/data.csv')\
        .save()


Ignore

PARQUET

In [0]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/Volumes/workspace/default/daya/csv/data.csv')\
.save()
     

TABLE

In [0]:
df.write.format('csv') \
  .mode('overwrite') \
  .option("header", "true") \
  .saveAsTable('my_table')

In [0]:
df.write.mode("overwrite").saveAsTable("my_table")

MANAGED VS EXTERNAL TABLES


SPARK SQL

createTempView

In [0]:
df.display()

SPARK SQL

In [0]:
df.createTempView('my_views')

In [0]:
%sql

select * from my_views where Item_Fat_Content = 'Low Fat'

In [0]:
%sql
select avg(Item_Weight) from my_views

In [0]:
df_sql = spark.sql("select * from my_view2 where Item_Fat_Content = 'Low Fat'")
df_sql.display()

In [0]:
%sql
select * from my_table 